[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abdulvajid1/LLM-FROM-SCRATCH/blob/master/Instruct_finetuning.ipynb)

In [6]:
!pip install -U datasets -q
!pip install -q tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.1 MB/s eta 0:00:00


You may requier restart the session if any load dataset get error

In [7]:
!git clone https://github.com/abdulvajid1/LLM-FROM-SCRATCH.git

Cloning into 'LLM-FROM-SCRATCH'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 153 (delta 74), reused 115 (delta 39), pack-reused 0 (from 0)
Receiving objects: 100% (153/153), 1.36 MiB | 9.32 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [8]:
%cd LLM-FROM-SCRATCH

/content/LLM-FROM-SCRATCH/LLM-FROM-SCRATCH


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset

from model import GPTModel
from config import Config
from load_weights_from_gpt import load_weights_into_gpt
from gpt_download import download_and_load_gpt2
from model_generate import generate
import tiktoken

from torch.utils.data import DataLoader
import os
from pathlib import Path

import glob

In [12]:
torch.manual_seed(0);

In [13]:
config = Config({
    "n_layers": 24,
    "d_model": 1024,
    "eps": 1e-5,
    "hidden_size_multiplier": 4,
    "num_heads": 16,
    "context_len": 1024,
    "dropout": 0.01,
    "qkv_bias": True,
    "vocab_size": 50257
})

In [14]:
torch.cuda.is_available()

False

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device : {device}")

device : cpu


In [16]:
tokenizer = tiktoken.get_encoding('gpt2')

In [17]:
model = GPTModel(config=config).to(device)

In [18]:
# total parameters
total_params = 0
for params in model.parameters():
    total_params += params.numel()

print(total_params)

406336593


In [19]:
settigns, params = download_and_load_gpt2(model_size='355M', models_dir='gpt2')

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 172kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 5.74MiB/s]
hparams.json: 100%|██████████| 91.0/91.0 [00:00<00:00, 200kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 1.42G/1.42G [00:56<00:00, 25.3MiB/s]
model.ckpt.index: 100%|██████████| 10.4k/10.4k [00:00<00:00, 16.2MiB/s]
model.ckpt.meta: 100%|██████████| 927k/927k [00:00<00:00, 4.56MiB/s]
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 2.94MiB/s]


In [20]:
load_weights_into_gpt(model, params)

In [21]:
print(generate(model,
               starting_context='fibonocci code in python: def fibonacci(n):',
               tokenizer=tokenizer,
               max_len=40))

fibonocci code in python: def fibonacci(n): for _ in range(20): yield n, 1 return fibonacci(1) Rehtaeh Baghi

10.0. 2014: Computerized fat tracking program for mathematics/


In [22]:
data = load_dataset('teknium/openhermes')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

openhermes.json:   0%|          | 0.00/328M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/242831 [00:00<?, ? examples/s]

In [23]:
data

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 242831
    })
})

In [24]:
data['train'][0]

{'output': '```perl\n#!/usr/bin/perl\n\nuse strict;\nuse warnings;\n\n# Check if a log file path is provided as command-line argument\nif (@ARGV != 1) {\n    print "Usage: $0 <log_file_path>\\n";\n    exit(1);\n}\n\nmy $log_file = $ARGV[0];\n\n# Open the log file for reading\nopen(my $fh, \'<\', $log_file) or die "Could not open file \'$log_file\' $!";\n\nmy %status_codes;\n\n# Process each line in the log file\nwhile (my $line = <$fh>) {\n    # Extract the HTTP status code from the log entry\n    if ($line =~ /HTTP\\/\\d+\\.\\d+" (\\d+)/) {\n        my $status_code = $1;\n        $status_codes{$status_code}++;\n    }\n}\n\nclose($fh);\n\n# Sort and print the results\nforeach my $code (sort { $status_codes{$b} <=> $status_codes{$a} } keys %status_codes) {\n    printf "%s: %d\\n", $code, $status_codes{$code};\n}\n```\n\nTo run this script, save it to a file named `count_http_status_codes.pl` and make sure it has executable permissions. Then, you can execute it with the following command

In [25]:
def apply_text_template(instr, input=''):
    template = """### Instruction: {instruction}
### Input: {input}
### Response:"""
    return template.format(instruction=instr, input=input)

In [26]:
def format_text_template(batch):
    template = """### Instruction: {instruction}
### Input: {input}
### Response: {output}"""

    return {'text': template.format(instruction=batch['instruction'], input=batch['input'], output=batch['output'])}

In [27]:
# testing map function
subset_data = data['train'].select([10,20,40])
subset_data = subset_data.map(format_text_template)

print(subset_data['text'][0])

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

### Instruction: Design a roller coaster with three distinct features, explaining the purpose of each feature and how it contributes to the overall ride experience.
### Input: 
### Response: 1. The Gravity-Defying Loop: One of the most iconic and thrilling features of our roller coaster is a massive, vertical loop that takes riders upside down as they travel through it at high speeds. This gravity-defying loop creates an intense sensation of weightlessness and disorientation for riders, making them feel like they are defying the laws of physics. As the train enters the loop, riders experience strong positive G-forces pushing them into their seats, followed by brief moments of weightlessness at the top of the loop before being pushed back into their seats again as they exit the loop. This feature contributes to the overall ride experience by providing a heart-pounding moment of adrenaline and excitement that leaves riders wanting more.

2. The Airtime Hills: To create a dynamic and vari

In [28]:
data = data.map(format_text_template, remove_columns=['output','input','instruction'])

Map:   0%|          | 0/242831 [00:00<?, ? examples/s]

In [29]:
# tokenize text in data
def tokenize(batch):
    token_ids = tokenizer.encode_batch(batch['text'], allowed_special={'<|endoftext|>'})
    return {'input_ids': token_ids}

In [30]:
dataset = data.map(tokenize, batched=True, remove_columns=['text'])

Map:   0%|          | 0/242831 [00:00<?, ? examples/s]

In [31]:
# filtering any datapoint which have more that 1024 tokens
dataset = dataset.filter(lambda x: len(x['input_ids']) <= 1024)

Filter:   0%|          | 0/242831 [00:00<?, ? examples/s]

In [32]:
tokenizer.decode(dataset['train'][0]['input_ids'])

'### Instruction: Write a Perl script that processes a log file and counts the occurrences of different HTTP status codes. The script should accept the log file path as a command-line argument and print the results to the console in descending order of frequency.\n\n### Input: \n### Response: ```perl\n#!/usr/bin/perl\n\nuse strict;\nuse warnings;\n\n# Check if a log file path is provided as command-line argument\nif (@ARGV != 1) {\n    print "Usage: $0 <log_file_path>\\n";\n    exit(1);\n}\n\nmy $log_file = $ARGV[0];\n\n# Open the log file for reading\nopen(my $fh, \'<\', $log_file) or die "Could not open file \'$log_file\' $!";\n\nmy %status_codes;\n\n# Process each line in the log file\nwhile (my $line = <$fh>) {\n    # Extract the HTTP status code from the log entry\n    if ($line =~ /HTTP\\/\\d+\\.\\d+" (\\d+)/) {\n        my $status_code = $1;\n        $status_codes{$status_code}++;\n    }\n}\n\nclose($fh);\n\n# Sort and print the results\nforeach my $code (sort { $status_codes{$b

In [33]:
dataset = dataset['train'].train_test_split(test_size=0.05, shuffle=True, seed=0)
train_dataset = dataset['train']

In [34]:
test_dataset = dataset['test'].train_test_split(test_size=0.4, seed=0)
val_dataset = test_dataset['train']
test_dataset = test_dataset['test']

In [35]:
tokenizer.eot_token

50256

In [36]:
def language_model_collate_fn(batch, max_len=None, tokenizer=tokenizer):
    # find max len seq in the batch
    pad_tok = tokenizer.eot_token

    batch = [seq['input_ids'] + [pad_tok] for seq in batch]

    batch_max_len = 0
    for seq in batch:
        if len(seq) > batch_max_len: # we give +1 since we have target token which need 1 token etc at end of token
            batch_max_len = len(seq)


    if max_len:
        max_len = max(batch_max_len, max_len)
    else:
        max_len = batch_max_len

    # truncate to max len
    batch = [seq[ :max_len] for seq in batch]

    # padding
    batch = [seq + [pad_tok] * (max_len - len(seq)) for seq in batch]

    # langauge modeling input_ids and output_ids
    input_ids = [seq[:-1] for seq in batch]
    target_ids = [seq[1:] for seq in batch]

    return torch.tensor(input_ids), torch.tensor(target_ids)

In [37]:
# # testing collate function
# language_model_collate_fn(batch=[[1,2,4,5,6],[2,3,3],[1]],
#                           tokenizer=tokenizer)

In [38]:
# initilaize dataloader
def create_dataloader(batch_size=4):
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                            collate_fn=language_model_collate_fn,
                            pin_memory=False, shuffle=True,
                            drop_last=True)

    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size,
                            collate_fn=language_model_collate_fn)

    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                            collate_fn=language_model_collate_fn)
    return train_loader, val_loader, test_loader

In [39]:
train_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 224148
})

In [40]:
test_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 4720
})

In [41]:
val_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 7078
})

In [42]:
train_loader, val_loader, test_loader = create_dataloader(batch_size=1)

In [43]:
# for input_ids, target_ids in train_loader:
#     break

In [44]:
# print(tokenizer.decode(input_ids[0].tolist()))

In [45]:
# model.eval();
# model.to(device);

In [46]:
# # check input to model
# with torch.no_grad():
#     output = model(input_ids[0].unsqueeze(0).to('cuda'))

In [47]:
# loss on batch
def calc_loss_on_batch(model, input_ids, target_ids):
    logits = model(input_ids).flatten(0, 1)
    target = target_ids.view(1, -1).squeeze()
    loss = torch.nn.functional.cross_entropy(logits, target)
    return loss

In [48]:
# cal loss on entire loader for eval
def cal_loss_on_data_loader(model, data_loader, max_batch=None):
    total_loss = 0
    total_batch = len(data_loader)

    if max_batch:
        max_batch = min(total_batch, max_batch)
    else:
        max_batch = total_batch

    for i, (input_ids, target_ids) in enumerate(data_loader):
        if i == max_batch:
            break
        input_ids, target_ids = input_ids.to(device), target_ids.to(device)
        loss = calc_loss_on_batch(model, input_ids, target_ids)
        total_loss += loss.item()

    return total_loss / max_batch


In [49]:
def evaluate(model, train_loader, val_loader, max_batch):
    model.eval()
    with torch.no_grad():
        train_loss = cal_loss_on_data_loader(model, train_loader, max_batch=max_batch)
        val_loss = cal_loss_on_data_loader(model, val_loader, max_batch=max_batch)
    model.train()
    return train_loss, val_loss

In [50]:
def generate(model,
            starting_context:str='what is the opposit of saying "i love you"',
            tokenizer=tokenizer,
            max_len=10,
            sampling=True,
            temperature=0.0,
            top_k=None,
            eos_id=None):


    starting_context = apply_text_template(instr=starting_context, input='')
    model.eval()
    input_ids = tokenizer.encode(starting_context)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)

    for i in range(max_len):
        with torch.no_grad():
            logits = model(input_ids)
            logits = logits[:,-1,:]

            if sampling:

                if top_k:
                    topk_logits, topk_pos = torch.topk(logits, k=top_k, dim=-1)
                    logits = torch.where(input=torch.tensor(float('-inf')),
                                         condition=logits < topk_logits[:,-1].reshape(-1, 1),
                                         other=logits)
                if temperature>0.0:
                    logits = logits / temperature

                probas = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probas, num_samples=1)
                input_ids = torch.concat([input_ids, idx_next], dim=-1)
            else:
                assert temperature==0.0 and top_k is None, "You can't set temperature or topk if sampling=False"
                last_token = torch.argmax(logits, dim=-1).unsqueeze(0)
                input_ids = torch.cat([input_ids, last_token], dim=-1)
    return tokenizer.decode(input_ids.squeeze().tolist())

In [51]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

In [52]:
# def train(model, train_loader, val_loader, optimizer, eval_freq, save_model_epoch_freq=30, save_file='./'):
#     global_step = 0
#     train_loss_history = []
#     val_loss_history = []
#     model.to(device)

#     for epoch, (input_ids, target_ids) in enumerate(train_loader):
#         model.train()
#         optimizer.zero_grad()
#         input_ids, target_ids = input_ids.to(device), target_ids.to(device)
#         loss = calc_loss_on_batch(model, input_ids, target_ids)
#         loss.backward()
#         optimizer.step()

#         global_step += 1

#         if global_step % eval_freq == 0 or epoch == 0:
#             train_loss, val_loss = evaluate(model, train_loader, val_loader, max_batch=10)
#             print(f"epoch: {epoch} | training_loss: {train_loss} | val_loss: {val_loss}")
#             train_loss_history.append(train_loss)
#             val_loss_history.append(val_loss)
#             print(f"Sample Generation\n{'-'*20}")
#             print(generate(model))


#     return train_loss_history,  val_loss_history



In [53]:
# train_losses, val_losses = train(model, train_loader, val_loader, optimizer, eval_freq=5, save_model_epoch_freq=10, save_file='checkpoints')

In [54]:
out = generate(model, starting_context='what is it')
print(out)

### Instruction: what is it
### Input: 
### Response: <MIME type="text/plain" content


In [55]:
# def train(model, train_loader, val_loader, optimizer, eval_freq, device, save_model_step_freq=10, save_file='runs', total_epoch=2):

#     model.to(device)

#     checkpoints_path = Path('runs')
#     checkpoints_path.mkdir(exist_ok=True)
#     model_checkpoints = list(checkpoints_path.glob("*.ckpt"))

#     if model_checkpoints:
#         last_save_path = model_checkpoints[-1].as_posix()
#         print(f'Loading last saved checkpoint : {last_save_path}')
#         training_state = torch.load(last_save_path)
#         global_step = training_state['global_step']
#         print(model.load_state_dict(training_state['model_state']))
#         optimizer.load_state_dict(training_state['optimizer_state'])
#         train_loss_history = training_state['train_loss_history']
#         val_loss_history = training_state['val_loss_history']
#         epoch = training_state['epoch']
#     else:
#         global_step = 0
#         train_loss_history = []
#         val_loss_history = []
#         epoch = 0

#     print(f'current global_step: {global_step}')
#     print(f'current epoch: {epoch}')
#     if train_loss_history:
#         print(f'last loss: {train_loss_history[-1]} | val_loss: {val_loss_history[-1]}')


#     for epoch in range(epoch, total_epoch):

#         for input_ids, target_ids in train_loader:
#             optimizer.zero_grad()
#             input_ids, target_ids = input_ids.to(device), target_ids.to(device)
#             loss = calc_loss_on_batch(model, input_ids, target_ids)
#             loss.backward()
#             optimizer.step()

#             global_step += 1

#             if global_step % (save_model_step_freq) == 0 :
#                 print('saving model...')
#                 torch.save({'model_state' : model.state_dict(),
#                             'optimizer_state': optimizer.state_dict(),
#                             'epoch': epoch,
#                             'global_step':global_step,
#                             'train_loss_history': train_loss_history,
#                             'val_loss_history': val_loss_history},
#                            (checkpoints_path / f"checkpoint.ckpt" ).as_posix())
#                 print('save completed..')

#             if global_step % eval_freq == 0 or epoch == 0:
#                 train_loss, val_loss = evaluate(model, train_loader, val_loader, max_batch=10)
#                 print(f"global_step: {global_step} | training_loss: {train_loss} | val_loss: {val_loss}")
#                 train_loss_history.append(train_loss)
#                 val_loss_history.append(val_loss)
#                 print(f"Sample Generation\n{'-'*20}")
#                 print(generate(model, starting_context="what is the opposit of word 'love'"))



#     return train_loss_history,  val_loss_history

In [56]:
from tqdm import tqdm
from pathlib import Path
import torch

def train(model, train_loader, val_loader, optimizer, eval_freq, device, save_model_step_freq=10, save_file='runs', total_epoch=2):

    model.to(device)

    checkpoints_path = Path('runs')
    checkpoints_path.mkdir(exist_ok=True)
    model_checkpoints = list(checkpoints_path.glob("*.ckpt"))

    if model_checkpoints:
        last_save_path = model_checkpoints[-1].as_posix()
        print(f'Loading last saved checkpoint : {last_save_path}')
        training_state = torch.load(last_save_path)
        global_step = training_state['global_step']
        print(model.load_state_dict(training_state['model_state']))
        optimizer.load_state_dict(training_state['optimizer_state'])
        train_loss_history = training_state['train_loss_history']
        val_loss_history = training_state['val_loss_history']
        epoch = training_state['epoch']
    else:
        global_step = 0
        train_loss_history = []
        val_loss_history = []
        epoch = 0

    print(f'Current global_step: {global_step}')
    print(f'Current epoch: {epoch}')
    if train_loss_history:
        print(f'Last loss: {train_loss_history[-1]} | Val loss: {val_loss_history[-1]}')

    for epoch in range(epoch, total_epoch):
        print(f"\n--- Epoch {epoch + 1}/{total_epoch} ---")
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=True)

        for input_ids, target_ids in progress_bar:
            optimizer.zero_grad()
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)
            loss = calc_loss_on_batch(model, input_ids, target_ids)
            loss.backward()
            optimizer.step()

            global_step += 1
            progress_bar.set_description(f"Epoch {epoch+1} | Step {global_step}")
            progress_bar.set_postfix(loss=loss.item())

            if global_step % save_model_step_freq == 0:
                print('\nSaving model...')
                torch.save({
                    'model_state': model.state_dict(),
                    'optimizer_state': optimizer.state_dict(),
                    'epoch': epoch,
                    'global_step': global_step,
                    'train_loss_history': train_loss_history,
                    'val_loss_history': val_loss_history
                }, (checkpoints_path / f"checkpoint.ckpt").as_posix())
                print('Save completed.')

            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate(model, train_loader, val_loader, max_batch=10)
                print(f"\n[Eval] Global Step: {global_step} | Training Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
                train_loss_history.append(train_loss)
                val_loss_history.append(val_loss)
                print(f"Sample Generation\n{'-'*20}")
                print(generate(model, starting_context="what is the opposite of word 'love'", sampling=False))

    return train_loss_history, val_loss_history


In [ ]:
train_losses, val_losses = train(model,
                                 train_loader,
                                 val_loader,
                                 optimizer,
                                 total_epoch=100,
                                 eval_freq=100,
                                 save_model_step_freq=100,
                                 save_file='runs',
                                 device=device)

Current global_step: 0
Current epoch: 0

--- Epoch 1/100 ---


Epoch 1 | Step 1:   0%|          | 1/224148 [00:27<1699:19:04, 27.29s/it, loss=2.13]

In [ ]:
question = "who is james bond?"

In [ ]:

print(generate(model, starting_context=question, max_len=50, temperature=0.7))

In [ ]:
print(generate(model, starting_context=question, max_len=50, sampling=False))